In [1]:
client_id = 
client_secret =

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 y6yylsn18y6hzdy4qqi2jvrr0cr8ju


In [2]:
oauth_token = 

In [3]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/spursito"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '104455653', 'login': 'spursito', 'display_name': 'Spursito', 'type': '', 'broadcaster_type': 'partner', 'description': 'Dosis frenéticas de diversión y alegría, pero en directo.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/01cafc14-0477-4f4d-a4d1-655058922ff6-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/4c8282fc-4143-42ab-9354-6fb38a7e4e41-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2015-10-15T22:26:30Z'}]}


In [4]:
data = {'data': [{'id': '104455653', 'login': 'spursito', 'display_name': 'Spursito', 'type': '', 'broadcaster_type': 'partner', 'description': 'Dosis frenéticas de diversión y alegría, pero en directo.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/01cafc14-0477-4f4d-a4d1-655058922ff6-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/4c8282fc-4143-42ab-9354-6fb38a7e4e41-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2015-10-15T22:26:30Z'}]}
data_dict = data['data'][0]
data_dict
id_canal =  list(data_dict.values())[0]
id_canal

'104455653'

In [5]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             17 non-null     int64         
 1   URL del VOD            17 non-null     object        
 2   Nombre del VOD         17 non-null     object        
 3   Duración               17 non-null     object        
 4   Fecha de inicio        17 non-null     datetime64[ns]
 5   Fecha de finalización  17 non-null     datetime64[ns]
 6   Visitas                17 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 9.1 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1887433629,https://www.twitch.tv/videos/1887433629,kings_league_🏆_especial_renovaciones_y_fichaje...,2h26m18s,2023-08-01 17:08:05,2023-08-01 17:08:05,171962
1,1885925482,https://www.twitch.tv/videos/1885925482,kings_league_🏆_los_descartes_❌_y_los_que_sigue...,2h9m18s,2023-07-30 20:45:12,2023-07-30 20:45:12,258991
2,1881557213,https://www.twitch.tv/videos/1881557213,queens_league_🏆_cuartos_de_final_☎️_rayo_de_ba...,1h3m22s,2023-07-25 20:16:09,2023-07-25 20:16:09,141077
3,1879671732,https://www.twitch.tv/videos/1879671732,kings_league_🏆_finales__y_debate_☎️_prince_cup...,3h19m8s,2023-07-23 16:24:18,2023-07-23 16:24:18,197407
4,1873509366,https://www.twitch.tv/videos/1873509366,rayo_de_barcelona_vs_porcinas_⚡_queens_leaguee...,1h1m10s,2023-07-16 16:00:22,2023-07-16 16:00:22,187765


In [6]:
df.to_csv('twitch_api_spursito_csv.csv', index=False)